# - Digital Signal Processing

Tacademy의 '디지털신호처리 이해'를 보고 정리

In [1]:
!pip install torch
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Digital Signal Processing

목적은 소리 signal을 어떠한 데이터 타입으로 표현하며, 소리와 관련된 task를 해결하는데 있음.

## Sound

소리는 일반적으로 진동으로 인한 공기의 압축으로 생성됨. 압축이 얼마나 됐느냐에 따라서 표현되는것이 Wave(파동). 파동은 진동하며 공간/매질을 전파해 나가는 현상임. 질량의 이동은 없지만 에너지/운동량의 운반은 존재

Wave에서 얻을수 있는 정보는 크게 3가지
- Phase(Degress of displacement) : 위상
- Amplitude : 진폭
- Frequency : 주파수

## Sampling

Contiuous 한 데이터를 잘게 쪼개서 discrete한 디지털 정보로 표현할 필요가 있음. 무한하게 쪼개서 저장할 수 없으니, 어떤 기준을 가지고 아날로그 정보를 쪼개서 대표값을 취하게 됨. 즉, Sampling을 한다는 말.

***Convert into a sequence of binary values via Sampling and Quantization***

## 1.1 Time domain

시간을 기준으로 아날로그 시그널을 쪼개게 되는것. Sampling을 통하여 컴퓨터는 소리 sequence를 binary value로 받아드리게됨.


### Sampling rate <- 설정값을 어떻게?

잘게 쪼갤수록 원본 데이터와 거의 가까워지기 때문에 좋지만 Data의 양이 증가하게 됨. 만약 너무 크게 쪼개게 된다면, 원본 데이터로 reconstruct하기 힘듬. 각각의 장단점이 존재


### Sampling theorem

샘플링 레이트가 최대 frequency의 2배 보다 커져야 한다는 것.<br/>
$f_s > 2f_m$ ($f_s$: sampling rate, $f_m$: maximum frequency)<br/>
- Nyqusit frequency = $f_s/2$, sampling rate의 절반

일반적으로 Sampling은 인간의 청각 영역에 맞게 형성
- Audio CD : 44.1 kHz(44100 sample/second)
- Speech communication : 8 kHz(8000 sample/second)

In [2]:
# Data preview
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchaudio

In [4]:
# tra
# train_dataset = torchaudio.datasets.LIBRISPEECH("./", url="train-clean-100", download=True)
test_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

  0%|          | 0.00/331M [00:00<?, ?B/s]

In [5]:
# data
print(type(test_dataset), len(test_dataset))
print(test_dataset[2])

print(test_dataset[2][4] / test_dataset[2][1]) # duration

<class 'torchaudio.datasets.librispeech.LIBRISPEECH'> 2620
(tensor([[ 0.0004,  0.0006,  0.0008,  ..., -0.0001, -0.0004, -0.0007]]), 16000, 'AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS', 1089, 134686, 2)
8.417875


2620개의 speech data<br/>
각각의 데이터는 다음과 같은 튜플로 구성되어있음.<br/>
(data(tensor 자료형), sampling rate, utterance, _, 전체길이, 인덱스)

In [7]:
test_dataset[1][0].shape # torch 자료형으로 0번 차원이 추가되어있음

torch.Size([1, 52400])

In [11]:
audio_data = test_dataset[1][0][0] # 0번 차원 제외시켜줌
sr = test_dataset[1][1] # sampling rate
print(audio_data, audio_data.shape)

tensor([ 0.0010,  0.0011,  0.0009,  ..., -0.0007, -0.0010, -0.0011]) torch.Size([52400])


In [14]:
len(audio_data) / sr # duration

3.275

In [16]:
test_dataset[1][2]

'STUFF IT INTO YOU HIS BELLY COUNSELLED HIM'

In [15]:
import IPython.display as ipd
ipd.Audio(audio_data, rate=sr)

In [17]:
# 16kHz -> 8kHz 2배 느려짐
ipd.Audio(audio_data, rate=8000)

In [18]:
# 16kHz -> 32kHz 2배 빨라짐
ipd.Audio(audio_data, rate=32000)

## Resampling

샘플링된 데이터를 더 높은 sampling rate 혹은 더 낮은 sampling rate로 다시 샘플링.<br/>
이때는 일반적으로 interpolation을 할때는 low-pass filter를 사용 (Windowed sinc function)

In [21]:
y_32k = librosa.resample(audio_data.numpy(), sr, 32000) # torch에서 바로 import해서tensor로 되어있어서 numpy의 array로 변경
ipd.Audio(y_32k, rate=32000)

In [23]:
y_8k = librosa.resample(audio_data.numpy(), sr, 8000)
ipd.Audio(y_8k, rate=8000)

In [27]:
print(audio_data.numpy()[:8])

print(y_8k[:8])
print(y_32k[:8])

print('16kHz:', len(audio_data.numpy()), '8kHz:', len(y_8k), '32kHz:', len(y_32k))

[0.0010376  0.00109863 0.00091553 0.00234985 0.0032959  0.00350952
 0.00204468 0.00088501]
[ 0.00066298  0.00136352  0.00319742  0.00233951 -0.00029481 -0.00242935
 -0.00051354  0.0018827 ]
[0.001049   0.00130799 0.00108487 0.00078758 0.00093091 0.00157119
 0.00233372 0.00291286]
16kHz: 52400 8kHz: 26200 32kHz: 104800


In [28]:
# resampling과정에서 sampling를 과하게 줄이면 복원이 잘되지않음
y_4k = librosa.resample(audio_data.numpy(), sr, 4000)
ipd.Audio(y_4k, rate=4000)

진폭을 수정했을때는 소리 크기에만 변화가 있을까?

In [32]:
y_np = audio_data.numpy()
y_am = y_np * 0.0000001
ipd.Audio(y_am, rate=sr)

변화가 없는거같음. 그래서 그런지 아래에서 scale 조정이나 값에 변화를 줬을때 음성이 그대로 들리는거같음.<br/>
자세한건 더 알아봐야될듯.

## Normalization & Quantization

시간의 기준이 아닌 실제 amplitude의 real valued를 기준으로 시그널의 값을 조절함. Amplitude를 이산적인 구간으로 나누고, signal 데이터의 Amplitude를 반올림.

이산적인 구간을 나눌때는 bit의 비트에 의해서 결정
- B bit의 Quantization : $-2^{B-1}$~$2^{B-1} - 1$
- Audio CD의 Quantization (16 bits) : $-2^{15}$~$2^{15} - 1$
- 위 값들은 보통 -1.0 ~ 1.0 영역으로 scaling 되기도 함

In [33]:
# Normalization
audio_np = audio_data.numpy()
normed_wav = audio_np / max(np.abs(audio_np)) # -1.0 ~ 1.0 사이값으로 변함
ipd.Audio(normed_wav, rate=sr)

In [34]:
# Quantization 하면 음질은 떨어지지만 light한 자료형이 됨
Bit = 8
max_value = 2 ** (Bit-1)

quantized_8_wav = normed_wav * max_value
quantized_8_wav = np.round(quantized_8_wav).astype(int)
quantized_8_wav = np.clip(quantized_8_wav, -max_value, max_value-1) # 최대최소값을 설정하고 그보다 위와 아래값을 최대최소값으로 변경
ipd.Audio(quantized_8_wav, rate=sr)

In [35]:
# norm-quanti 한번에
def norm_quanti(x, bit):
    # normalization
    x = x / max(np.abs(x))

    # quantization
    max_value = 2 ** (bit-1)

    q_x = x * max_value
    q_x = np.round(q_x).astype(int)
    q_x = np.clip(q_x, -max_value, max_value-1)

    return q_x

quantized_8_wav = norm_quanti(audio_np, 8)
ipd.Audio(quantized_8_wav, rate=sr)

print(max(quantized_8_wav), min(quantized_8))